In [2]:
import os
import sys

# Make sure we're pointing at the repo root
repo_root = os.path.abspath("..") if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

from forgeflow import paths

print(paths.get_project_root())
print(paths.get_data_dir("raw"))



/workspaces/forgeflow-manufacturing-analytics
/workspaces/forgeflow-manufacturing-analytics/data/raw
